In [3]:
import pprint
import os
import subprocess
import pandas as pd
import requests
import certifi
import ssl
import json
from scidownl import scihub_download
from pdftextract import XPdf
import glob
import re
import string
import json
import asyncio
from multiprocessing import Pool
#import nest_asyncio
#nest_asyncio.apply()
#__import__('IPython').embed()
sess = requests.Session()
ssl._create_default_https_context = ssl._create_unverified_context

In [116]:
pmid_df = pd.DataFrame()
doi_error = pd.DataFrame()
res_pmid = []
res_doi = []
res_output = []
res_doi_error = []
df = pd.read_table('../data/doi', header= None)
df.columns = ['Doi']
for i in df['Doi']:
    p = subprocess.Popen(f'convert doi2pmid {i}', shell= True, stdout=subprocess.PIPE)
    output = b''.join(p.stdout).decode('utf-8')
    if output.split('\n')[1] == 'PMID:  None\r' or output.split('\n')[1] == 'PMID:  NOT_FOUND;INVALID_JOURNAL\r' or output.split('\n')[1] == '':
        res_doi_error.append(output.split('\n')[0])
    else:
        res_doi.append(output.split('\n')[0])
        res_pmid.append(output.split('\n')[1])
pmid_df['DOI'] = res_doi
pmid_df['PMD'] = res_pmid
doi_error['DOI'] = res_doi_error
pmid_df.to_csv('../data/PMD.csv')
doi_error.to_csv('../data/DOI_ERROR.csv')

In [198]:
#Clean PMID
def load_data():
    all_table = pd.read_csv('../data/PMD.csv')
    pmd = all_table['PMD']
    pmd = pmd.str.replace('PMID:', '')
    pmd = pmd.str.replace('\r', '')
    pcmd = pmd.str.replace("    ", '')
    return pcmd

In [239]:
#Create pmid string
def create_pmid_req():
    pmid = []
    pmd = load_data()
    for i in pmd:
        pmid.append(i)
    pmid = ','.join(str(x) for x in pmid)
    pmid = pmid.replace('  ','')
    return pmid


In [222]:
#Bern function add pmid
def bern(pmid):
    res = []
    URL = f"https://bern.korea.ac.kr/pubmed/{pmid}"
    r = sess.get(URL, verify=False)
    res.append(r)
    print(len(res))
    with open('bert.txt', 'w') as json_file:
        json.dump(r.json(), json_file)



In [240]:
#Load journal
def download_multi_papers():
    pmid = create_pmid_req()
    source = []
    for i in pmid.split(','):
        source.append((i, 'pmid', '../paper/'))

    for paper, paper_type, out in source:
        scihub_download(paper, paper_type=paper_type, out=out)


In [ ]:
#Run load journal
if __name__ == '__main__':
    download_multi_papers()

In [15]:
def query_raw(text, url="https://bern.korea.ac.kr/plain"):
    return sess.post(url, data={'sample_text': text}, verify=False).json()



In [18]:
def export_txt_bern():
    list_pdf = glob.glob('../paper/*pdf')
    try:
        for p in list_pdf:
            txt = extract_text_pdf(p)
            count = len(txt)
            while count >0:
                response = query_raw(txt[0:150])
                count -= 150
                if count <= 0:
                    break
                with open('data.json', 'w') as f:
                        json.dump(response, f)
    except:
        print('Failed response')





In [20]:
export_txt_bern()

122231


C:\Users\dbair\anaconda3\envs\Proteins\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'bern.korea.ac.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\dbair\anaconda3\envs\Proteins\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'bern.korea.ac.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\dbair\anaconda3\envs\Proteins\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'bern.korea.ac.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\dbai

Failed response


In [19]:
def extract_text_pdf(file):
    pdf = XPdf(file)
    txt_2 = ''

    txt = pdf.to_text()
    txt = txt.strip('\r\n')
    #txt = txt.replace('\r', '')
    #txt = txt.replace('\n', '')
    txt = re.sub(r'\d+', '', txt)
    txt = txt.translate(str.maketrans('', '', string.punctuation))
    txt = txt.strip('\t')
    txt = ' '.join(txt.split())
    x = "●"
    y = " "
    table = txt.maketrans(x, y)
    txt = txt.translate(table)
    print(len(txt))
    return txt


In [5]:
extract_text_pdf('../paper/A time-resolved proteomic and prognostic map of COVID-19. Cell Systems, 12(8), 780–794.e7.pdf')

122231


'Article A timeresolved proteomic and prognostic map of COVID Graphical abstract Connecting dynamic changes Authors in plasma proteome and Longitudinal profiling of samples clinical parameters Vadim Demichev Pinkus ToberLau from COVID inpatients Oliver Lemke PACOVID Study group Markus Ralser Florian Kurth plasma samples clinical Correspondence parameters markusralsercharitede cell counts ELISA In brief enzyme activity Demichev ToberLau et al present a sensitivityMachine learning models link timeresolved molecular map of the phenotype to measurements COVID measuring plasma proteomes of patients with COVID along with an mechanical ventilation extensive panel of clinical diagnostic parameters at time points They Proteome describe the speciﬁcity and dynamics as Age well as the predictive and prognostic BMI power of the molecular signatures in CCI COVID IL CRP specificity Highlights d Plasma proteomes combined with clinical parameters characterize COVID progression d Machine learning models